In [72]:
import numpy as np
import pandas as pandas
import matplotlib.pyplot as plt

In [73]:
rute='/home/ubuntu/Tesis/Results/Tesis/PerformanceNewPrompt'
file='TestJanusAll5.csv' 
df=pandas.read_csv(rute+'/'+file, sep=',', header=0)
print(df)

             Name  Mode  True Positive  False Positive  False Negative  \
0     4_076_1.mp4     0             16               0               0   
1     3_106_1.mp4     0             19               0               0   
2     1_067_1.mp4     0             15               0               0   
3     1_066_1.mp4     0             16               0               0   
4     2_096_1.mp4     0             29               0               0   
...           ...   ...            ...             ...             ...   
1255  D148_04.avi     4              2             116              42   
1256  D149_05.avi     4             12               1              18   
1257  D158_13.avi     4              5               0              12   
1258  D149_07.avi     4              3               3              21   
1259  D149_03.avi     4              7               6              70   

      True Negative                               True Event  \
0                 0                a person rid

In [74]:
def metrics_from_df(df):
    mode_names = {
            0: "Detector con reglas, MLLM e información",
            1: "MMLM Solo",
            2: "Detector, MLLM e informacion",
            3: "Detector con reglas y MLLM",
            4: "Detector con reglas ",
        }
    df.rename(columns={'True Event': 'Event', "True Positive": "TP", "False Positive": "FP", "False Negative": "FN", "True Negative": "TN"}, inplace=True)
    df_sum = df.groupby(["Event", "Mode"])[ ["TP", "FP", "FN", "TN"]].sum().reset_index()

    # Initialize precision, recall, accuracy with zeros
    df_sum["Precision"] = 0.0
    df_sum["Recall"] = 0.0
    df_sum["Accuracy"] = 0.0

    # Calculate only where the denominator is not zero
    mask_prec = (df_sum["TP"] + df_sum["FP"]) != 0
    df_sum.loc[mask_prec, "Precision"] = (
        df_sum.loc[mask_prec, "TP"] / (df_sum.loc[mask_prec, "TP"] + df_sum.loc[mask_prec, "FP"])
    )

    mask_recall = (df_sum["TP"] + df_sum["FN"]) != 0
    df_sum.loc[mask_recall, "Recall"] = (
        df_sum.loc[mask_recall, "TP"] / (df_sum.loc[mask_recall, "TP"] + df_sum.loc[mask_recall, "FN"])
    )

    mask_acc = (df_sum["TP"] + df_sum["FP"] + df_sum["FN"] + df_sum["TN"]) != 0
    df_sum.loc[mask_acc, "Accuracy"] = (
        (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "TN"]) /
        (df_sum.loc[mask_acc, "TP"] + df_sum.loc[mask_acc, "FP"] + df_sum.loc[mask_acc, "FN"] + df_sum.loc[mask_acc, "TN"])
    )
    df_sum.drop(columns=['TP', 'FP', 'FN', 'TN'], inplace=True)
    df_sum.drop(columns=['Event'], inplace=True)
    df_sum=df_sum.groupby(["Mode"]).mean()
    df_sum.rename(index=mode_names, inplace=True)
    df_sum['F1'] = 2 * df_sum['Precision'].astype(float) * df_sum['Recall'].astype(float) / (df_sum['Precision'].astype(float) + df_sum['Recall'].astype(float))
    df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
    return df_sum

In [75]:
metrics=metrics_from_df(df)
print(metrics)

                                         Precision    Recall  Accuracy  \
Mode                                                                     
Detector con reglas, MLLM e información   0.660884  0.999204  0.661164   
MMLM Solo                                 0.538725  0.771577  0.612019   
Detector, MLLM e informacion              0.453356  0.973084  0.485356   
Detector con reglas y MLLM                0.741165  0.801464  0.721844   
Detector con reglas                       0.663671  0.320521  0.606194   

                                               F1  
Mode                                               
Detector con reglas, MLLM e información  0.795570  
MMLM Solo                                0.634461  
Detector, MLLM e informacion             0.618537  
Detector con reglas y MLLM               0.770136  
Detector con reglas                      0.432274  


/tmp/ipykernel_8334/835913274.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sum['F1'].fillna(0, inplace=True)  # Handle division by zero
